In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import RobustScaler
import random
import tensorflow as tf
import re
import os

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
#temp,dew,humidity,cloud_cover,solar_radiation,solar_energy, uv_radiation
# Define initial min and max values
min_val = [1000, 1000, 1000, 1000, 1000, 1000, 1000]
max_val = [-1000, -1000, -1000, -1000, -1000, -1000, -1000]

In [ ]:
def adjust_dataframe_length(df, target_length):
    # Check if df.shape[0] is less than the target length
    if df.shape[0] < target_length:
        df = pd.concat([df, df.tail(target_length - df.shape[0])])

    # Check if df.shape[0] is greater than the target length
    if df.shape[0] > target_length:
        # Create a list of rows where the 4th column (precip) value is 0
        zero_precip_rows = []
        for i in range(len(df)):
            # if df.iloc[i, 3] == 0:
            zero_precip_rows.append(i)

        # Randomly select rows from the list of zero_precip_rows
        rows_to_remove = random.sample(zero_precip_rows, df.shape[0] - target_length)

        # Remove the selected rows from the DataFrame
        df = df.drop(rows_to_remove)
    return df

# Normalize the data using Min-Max scaling
def min_max_scaling(df, min_val, max_val):
    for i, col in enumerate(df.columns):
        df[col] = (df[col] - min_val[i]) / (max_val[i] - min_val[i])
    return df

def append_rows_to_single_row(df):
    # Normalize each column separately
    # Normalize the data using Min-Max scaling
    df_normalized = min_max_scaling(df, min_val, max_val)

    # Create an empty dictionary to store the data
    data_dict = {}

    # Number of days
    num_days = len(df_normalized)

    # Iterate over each row
    for i in range(num_days):
        # Get the data from the current row
        row_data = df_normalized.iloc[i].values

        # Append the data to the dictionary with modified column names
        for j, value in enumerate(row_data):
            column_name = f"{df_normalized.columns[j]}_day_{i+1}"
            data_dict[column_name] = value

    # Convert the dictionary to a DataFrame
    single_row_df = pd.DataFrame([data_dict])

    return single_row_df

def calculate_min_max(df):
    global min_val, max_val
    for i, col in enumerate(df.columns):
        min_val[i] = min(df[col].min(), min_val[i])
        max_val[i] = max(df[col].max(), max_val[i])


def load_and_adjust_data(urls, target_length):

    dfs = []

    for url in urls:
        # Read Excel file into DataFrame
        df = pd.read_excel(url)
        # Fill NaN values with column means
        df= df.drop(columns=['precip'])
        df = df.fillna(df.mean())
        if df.isna().any().any():
            print("NaN values encountered in DataFrame from URL:", url)
        #calculating for min max scaling
        calculate_min_max(df)
        # Adjust the length of the DataFrame
        df = adjust_dataframe_length(df, target_length)
        # Append the adjusted DataFrame to the list
        dfs.append(df)
    return dfs

def extract_label_from_url(url):

    filename = url.split('/')[-1]  # Get the filename from the URL
    label = re.split('_\d+', filename)[1]  # Extract the label based on the pattern
    return label[1:]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
mydrive_path = "/content/drive/MyDrive/ACPS_Project/Modified for model"

# List files in MyDrive with full paths
urls = [os.path.join(mydrive_path, file) for file in os.listdir(mydrive_path)]
print(urls)

In [ ]:
labels = []
for url in urls:
  labels.append(extract_label_from_url(url))
# Load and adjust the data
dfs = load_and_adjust_data(urls, 180)

# Convert each DataFrame to a single row
single_row_dfs = [append_rows_to_single_row(df) for df in dfs]

# Concatenate the single-row DataFrames
df_concatenated = pd.concat(single_row_dfs, axis=0)

In [ ]:
print(min_val)
print(max_val)

In [ ]:
# Check if there are any NaN values in the DataFrame
if df_concatenated.isna().any().any():
    # Print the rows with NaN values
    print("Rows with NaN values:")
    print(df_concatenated[df_concatenated.isna().any(axis=1)])
# Drop rows with NaN values
df_concatenated = df_concatenated.dropna()

# Print the DataFrame after dropping rows
print(df_concatenated)


In [ ]:
#Split the data into train and test sets
X_train, X_test = train_test_split(df_concatenated.values, test_size=0.2, random_state=42)

# Further split the training data into train and validation sets
X_train, X_val = train_test_split(X_train, test_size=0.1, random_state=42)

# Reshape the data
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)

# Define the input layers
input_layer_train = Input(shape=(X_train.shape[1], X_train.shape[2]))
input_layer_test = Input(shape=(X_test.shape[1], X_test.shape[2]))
input_layer_val = Input(shape=(X_val.shape[1], X_val.shape[2]))

conv1 = Conv1D(512,kernel_size=3,activation='relu')(input_layer_train)
conv2 = Conv1D(512,kernel_size=3,activation='relu')(conv1)
conv3 = Conv1D(128,kernel_size=5,activation='relu')(conv2)
conv4 = Conv1D(128,kernel_size=5,activation='relu')(conv3)
flatten = Flatten()(conv4)

# Define the encoding layers
encoding_layer1 = Dense(512, activation='relu')(flatten)
encoding_layer2 = Dense(256, activation='relu')(encoding_layer1)
encoding_layer3 = Dense(128, activation='relu')(encoding_layer2)

# Define the decoding layers
#decoding_layer1 = Dense(128, activation='relu')(encoding_layer3)
decoding_layer2 = Dense(256, activation='relu')(encoding_layer3)
decoding_layer3 = Dense(512, activation='relu')(decoding_layer2)
decoding_layer4 = Dense(X_train.shape[1], activation='relu')(decoding_layer3)

# Define the autoencoder model
autoencoder = Model(inputs=input_layer_train, outputs=decoding_layer4)

# Compile the autoencoder
autoencoder.compile(optimizer='adam', loss='mean_absolute_error')

# Train the autoencoder on the training data
history = autoencoder.fit(X_train, X_train, epochs=1024, validation_data=(X_val, X_val))


# Evaluate the autoencoder on the test set
reconstruction_error_test = autoencoder.evaluate(X_test, X_test)
print("Reconstruction error on test set:", reconstruction_error_test)


In [ ]:
# Plot the training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Plot the reconstruction error for both training and test sets
reconstruction_error_train = autoencoder.evaluate(X_train, X_train)
print("Reconstruction error on training set:", reconstruction_error_train)
# Evaluate the autoencoder on the test set
reconstruction_error_test = autoencoder.evaluate(X_test, X_test)
print("Reconstruction error on test set:", reconstruction_error_test)
plt.bar(['Training', 'Test'], [reconstruction_error_train, reconstruction_error_test])
plt.title('Reconstruction Error')
plt.ylabel('Loss')
plt.show()

In [ ]:
labels = np.array(labels)
unique_labels = np.unique(labels)
print(unique_labels)

In [ ]:
# Map each label to a numerical value
label_mapping = {'jowar': 0, 'rice': 1, 'wheat': 2, 'arhar':3, 'bajra':4, 'barley':5, 'gram':6,'maize':7, 'ragi':8, 'urad':9}

# Convert labels to numerical values
numerical_labels = np.array([label_mapping[label] for label in labels])

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df_concatenated.values, numerical_labels, test_size=0.2, random_state=42)

# Further split the training data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

# Define the encoder model
encoder_model = Model(inputs=input_layer_train, outputs=encoding_layer3)

# Extract features from the training, validation, and test sets
encoded_train = encoder_model.predict(X_train)
encoded_val = encoder_model.predict(X_val)
encoded_test = encoder_model.predict(X_test)

# Now you can use these encoded representations for further classification tasks
inputs = Input(shape=(encoded_train.shape[1],))
x = Dense(64, activation='relu')(inputs)
xy = Dense(32, activation='relu')(x)
outputs = Dense(10, activation='softmax')(xy)

classifier_model = Model(inputs=inputs, outputs=outputs)

# Compile the classifier model
classifier_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the classifier model
classifier_history = classifier_model.fit(encoded_train, y_train, epochs=200, validation_data=(encoded_val, y_val))

# Evaluate the classifier model on the test set
classifier_loss, classifier_accuracy = classifier_model.evaluate(encoded_test, y_test)
print("Classifier accuracy on test set:", classifier_accuracy)


In [ ]:
# Plot training loss
plt.plot(classifier_history.history['loss'], label='Training Loss')
# Plot validation loss
plt.plot(classifier_history.history['val_loss'], label='Validation Loss')
plt.title('Classifier Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plot training accuracy
plt.plot(classifier_history.history['accuracy'], label='Training Accuracy')
# Plot validation accuracy
plt.plot(classifier_history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Classifier Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Make predictions on the test data
predictions = classifier_model.predict(encoded_test)

# Convert predicted probabilities into class labels
predicted_labels = np.argmax(predictions, axis=1)

accuracy = np.mean(predicted_labels == y_test)

print("Accuracy:", accuracy)


In [ ]:
def predict_categories_from_urls(urls, encoder_model, classifier_model):
    # Load and adjust the data
    datas = load_and_adjust_data(urls, 180)

    # Convert each DataFrame to a single row
    single_row_datas = [append_rows_to_single_row(data) for data in datas]

    # Concatenate the single-row DataFrames
    datas_concatenated = pd.concat(single_row_datas, axis=0)

    # Use the trained encoder model to extract features from the reconstructed data
    encoded_reconstructed_data = encoder_model.predict(datas_concatenated.values.reshape(datas_concatenated.shape[0], datas_concatenated.shape[1], 1))

    # Make predictions on the reconstructed data using the classifier model
    predictions = classifier_model.predict(encoded_reconstructed_data)

    # Convert predicted probabilities into class labels
    predicted_labels = np.argmax(predictions, axis=1)

    # Map the class labels to their corresponding categories
    category_mapping = {0: 'jowar', 1: 'rice', 2: 'wheat',3:'arhar',4: 'bajra',5: 'barley',6: 'gram',7: 'maize',8: 'ragi',9:'urad'}
    predicted_categories = [category_mapping[label] for label in predicted_labels]

    return predicted_categories


In [ ]:
test_urls = ['/content/drive/MyDrive/ACPS_Project/Modified for model/Adilabad_2015_wheat_10-05_weather.xlsx']
predicted_categories = predict_categories_from_urls(test_urls, encoder_model, classifier_model)
print("Predicted categories:", predicted_categories)


In [ ]:
encoder_model.summary()

In [ ]:
classifier_model.summary()

In [ ]:
autoencoder.summary()

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model

# Assuming your model is already defined and trained
model = classifier_model
automodel = encoder_model
# model.save('/content/my_model/classifier.h5')
automodel.save('/content/drive/MyDrive/encoder_model.h5')

In [ ]:
import pandas as pd

def excel_to_json(excel_file_path):
    # Read Excel file into DataFrame
    df = pd.read_excel(excel_file_path)

    # Drop the 'precip' column
    df.drop(columns=['precip'], inplace=True)

    # Fill NaN values with column means
    df.fillna(df.mean(), inplace=True)

    # Convert DataFrame to JSON
    json_data = df.to_json(orient='records',force_ascii=False)

    return json_data

excel_file_path = '/content/drive/MyDrive/ACPS_Project/Modified for model/Adilabad_2015_wheat_10-05_weather.xlsx'
json_data = excel_to_json(excel_file_path)
print(json_data)

In [ ]:
pip install visualkeras

In [ ]:
import visualkeras

In [ ]:
!pip install git+https://github.com/paulgavrikov/visualkeras

In [ ]:
from PIL import ImageFont

font = ImageFont.load_default(128)

In [ ]:
import visualkeras

model = classifier_model

visualkeras.layered_view(model,draw_volume=False, legend=True).show() # display using your system viewer
visualkeras.layered_view(model, to_file='output.png', legend=True, font =font) # write to disk
visualkeras.layered_view(model, to_file='output.png', legend=True).show() # write and show